In [9]:
import tensorflow as tf
import numpy as np
from numpy.random import randn
import pathlib
import random
from PIL import Image
from matplotlib.image import imread
from keras.preprocessing import image
import matplotlib.pyplot as plt
import pandas as pd
import os

In [3]:
data_dir = pathlib.Path("/workspaces/awesome-ml-group/00_source_data/images/train")

In [4]:
label_names={'angry':0,
            'disgust':1,
            'fear':3,
            'happy':4,
            'neutral':5,
            'sad':6,
            'suprise':7
}

In [ ]:
img_data=[]
count=0
all_images = list(data_dir.glob('*'))
all_labels=[]

wrong_emotion=['disgust','fear','surprise','fea','angry']
for i in all_images:
  a=str(i)[str(i).rfind('/')+1:]
  emotion=a[:a.find('_')]

  if emotion not in wrong_emotion :
    img=Image.open(str(i))
    

    img=img.resize((64,64))
    img=np.array(img).astype('float64')
    img=img/255
    img_data.append(img)

In [35]:
img_data={}
all_image = list(data_dir.glob('*'))

# wrong_emotion=['disgust','fear','surprise','fea','angry']
for folder in all_image:
    a=str(folder)[str(folder).rfind('/')+1:]
    if a in label_names:
        img_data[a] = []
    else:
        continue
    for i in os.listdir(folder):
        i = str(folder)+'/'+i
        img=Image.open(i)
        img=img.resize((64,64))
        img=np.array(img).astype('float64')
        img=img/255
        img_data[a].append(img)

In [36]:
for k,v in img_data.items():
    print(k, len(v))

sad 4938
fear 4103
happy 7164
disgust 436
neutral 4982
angry 3993


In [37]:
#Masking
top_mask=[]
bottom_mask=[]

def creating_mask():
    a=np.array([0]*2048)
    a=a.reshape((32,64))
    b=np.array([1]*2048)
    b=b.reshape((32,64))
    return a,b


for i in img_data["sad"]:
    a,b=creating_mask()
    con_bot = np.concatenate((b,a),axis=0)
    con_top = np.concatenate((a,b),axis=0)
    bottom_mask.append(i[:32,:])
    top_mask.append(i[32:,:])